In [1]:
from lib_for_data_reading import *
from country_run import *
from perc import *
from lib_for_cc import *
import re
import numpy as np
import pandas as pd

In [2]:
nameofthisround='spring2020'


# directories
model             = os.getcwd()
data              = model+'/data/'
scenar_folder     = model+'/scenar_def/'
finalhhdataframes = model+'/finalhhdataframes/'
ssp_folder        = model+'/ssp_data/'
pik_data          = model+'/pik_data/'
iiasa_data        = model+'/iiasa_data/'

# simulation inputs
codes         = read_csv('wbccodes2014.csv')
codes_tables  = read_csv(ssp_folder+"ISO3166_and_R32.csv")
ssp_pop       = read_csv(ssp_folder+"SspDb_country_data_2013-06-12.csv",low_memory=False)
ssp_gdp       = read_csv(ssp_folder+"SspDb_compare_regions_2013-06-12.csv")
hhcat         = read_csv(scenar_folder+"hhcat_v2.csv")
industry_list = read_csv(scenar_folder+"list_industries.csv")

present    = "{}/{}_present/".format(model,nameofthisround)
baselines  = "{}/{}_baselines/".format(model,nameofthisround)
with_cc    = "{}/{}_with_cc/".format(model,nameofthisround)
baselines_2050  = "{}/{}_baselines_2050/".format(model,nameofthisround)
with_cc_2050    = "{}/{}_with_cc_2050/".format(model,nameofthisround)

In [3]:
######################################
# Load necessary files
#
#load climate change impacts data 
# ^ This will be IIASA inputs
price_increase_reg = read_csv(iiasa_data+"boundaries_food2.csv")
food_share_data = read_csv(data+"food_shares_wbreg.csv")
#
####
# Now WITH malaria!!!
malaria_diff    = read_csv(data+"malaria_diff_2006-2035.csv")
malaria_bounds  = read_csv(data+"malaria_bounds.csv",index_col="var")
diarrhea_bounds = read_csv(data+"diarrhea_bounds.csv",index_col="var")
#
#####
diarrhea_shares  = read_csv(data+"diarrhea_who.csv",index_col="Region")
diarrhea_shares.index=diarrhea_shares.index.str.strip()
diarrhea_reg    = read_csv(data+"wbccodes2014_who.csv",index_col="country")
#
#####
# No natural disasters!
fa                       = read_csv(data+"disasters/av_fa_for_cc_model.csv",index_col='iso3')
vulnerabilities          = read_csv(data+"disasters/vulnerabilities.csv",index_col='iso3')
#
#####
# Need to see what these are, as distinct from price_increase. Probably temperature?
climate_param_bounds     = read_csv(data+"climate_param_bounds.csv",index_col="var")
climate_param_thresholds = read_csv(data+"climate_param_thresholds.csv",index_col="var")
#
#
# No Stunting!
shares_stunting = None #read_csv(data+"share_stunting_who.csv")


######################################
# Get list of available countries...
list_csv=os.listdir(finalhhdataframes)
all_surveys=dict()
for myfile in [_ for _ in list_csv if 'DS_Store' not in _ and _ != '_Final_CPI_PPP_to_be_used.csv']:
    if 'finalhhframe' in myfile:
        cc = re.search('(.*)_finalhhframe.csv', myfile).group(1)
        all_surveys[cc]=read_csv(finalhhdataframes+myfile)

In [4]:
finalhhdataframes+myfile

'C:\\Users\\bjafino\\Documents\\GitHub\\PovertyClimateSpring2020/finalhhdataframes/_Final_CPI_PPP_to_be_used.dta'

In [5]:
####
#load codes, population
codes         = read_csv('wbccodes2014.csv')
codes_tables  = read_csv(ssp_folder+"ISO3166_and_R32.csv")
ssp_pop       = read_csv(ssp_folder+"SspDb_country_data_2013-06-12.csv",low_memory=False)
ssp_gdp       = read_csv(ssp_folder+"SspDb_compare_regions_2013-06-12.csv")
hhcat         = read_csv(scenar_folder+"hhcat_v2.csv")
industry_list = read_csv(scenar_folder+"list_industries.csv")
#
####
# Drivers of scenarios
uncertainties    = ['shareag','sharemanu','shareemp', 'shareurban',
                    'grserv','grag','grmanu','skillpserv','skillpag','skillpmanu','p','b','voice']
lhssample        = read_csv(scenar_folder+"lhs-table-500-13uncertainties.csv")
ranges = DataFrame(columns=['min','max'],index=uncertainties)

impact_scenars = read_csv(scenar_folder+"impact-scenarios-low-high.csv").tail(1)
# ^ IIASA here? 

######################################
# Packaging for country_run()
#
datalist=(hhcat,industry_list,codes_tables,lhssample,ranges,ssp_pop,ssp_gdp,food_share_data,impact_scenars,price_increase_reg,malaria_bounds, 
          diarrhea_bounds, shares_stunting, malaria_diff, diarrhea_shares, diarrhea_reg, climate_param_bounds, climate_param_thresholds,codes,fa,vulnerabilities)
#
ssp_to_test   = [5]
#

######################################
# Do the thing (TM)
#_record_keeping = read_csv('results_summary/GMD_to_finalhhframe_status.csv').set_index('country')
#for _c in ['BAU_complete','CC_complete']:
#    if _c not in _record_keeping.columns: _record_keeping[_c] = None

#### Do for one country

In [9]:
countrycode = 'BGD'
ini_year = 2011 #get_ini_year(countrycode)
data2day = 30
year = 2030
povline = 1.9

paramvar=(year,ini_year,data2day,ssp_to_test,povline)
scenar = 30
b_value = 0
switches = []
switches = ['all']
future_with_cc = True

In [10]:
future_with_cc

True

In [12]:
forprim_now, forprim_bau,forprim_cc, inc = country_run(countrycode,scenar,datalist,paramvar,all_surveys,
                                                  switches,b_value,future_with_cc, with_person_data=False)

In [54]:
# forprim_bau

In [55]:
# forprim_cc

In [56]:
# forprim_now

In [57]:
# inc

#### Do for all countries

In [9]:
df_master = pd.DataFrame()
not_working = []

ini_year = 2015 #get_ini_year(countrycode)
data2day = 30
year = 2030
povline = 1.9
paramvar=(year,ini_year,data2day,ssp_to_test,povline)
scenar = 18
b_value = 0
switches = []
switches = ['all']
future_with_cc = True

for country in list(all_surveys.keys()):
    print('Starting for ' + country)
    countrycode = country
    try:
        forprim_now, forprim_bau,forprim_cc, inc = country_run(countrycode,scenar,datalist,paramvar,all_surveys,
                                                               switches,b_value,future_with_cc, with_person_data=False)
        df = pd.DataFrame(inc).T
        df['country'] = country
        df_master = df_master.append(df)
    except:
        not_working.append(country)

Starting for AGO
27.90521240234375 19.25885009765625 1.0
Step 2 complete
2015 to 2030
Step 3 complete
Initial pop description:
         old   children  unemployed skillworkers  servworkers    agworkers  manuworkers        urban
pop  734703  1.519e+07  4.8293e+06  7.13706e+06  1.05139e+06  4.06309e+06  3.91514e+06  1.81284e+07
share employed_i: 0.6515384262634101 
share ag_i: 0.44997302703441167 
share manu_i: 0.4335886376109811 
share skilled_i: 0.5149797528422408 
share urban_i: 0.6086696184504928


C:\Users\bjafino\anaconda3\envs\py36wb\lib\site-packages\pandas\core\frame.py:6701: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


SSP/population growth (2030) 30013947.56 0.020818075692211835


C:\Users\bjafino\anaconda3\envs\py36wb\lib\site-packages\pandas\util\_decorators.py:188: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)




Fit parameter inputs (ranges):
                   min        max
shareag      0.154928   0.479329
sharemanu    0.166678    0.39474
shareemp     0.554211   0.819333
shareurban    0.70408    0.70408
grserv     -0.0309468  0.0290532
grag       -0.0309468  0.0290532
grmanu     -0.0309468  0.0290532
skillpserv     2.3362     2.3362
skillpag      1.98006    1.98006
skillpmanu    1.70963    1.70963
p               0.001        0.2
b                   0          0
voice               0          1
Step 5 complete


Running mosek fit with these input values:
 shareag       0.317129
sharemanu     0.280712
shareemp      0.686764
shareurban    0.704080
grserv       -0.000948
grag         -0.000945
grmanu       -0.000946
skillpserv    2.336201
skillpag      1.980056
skillpmanu    1.709626
p             0.100509
b             0.000000
voice         0.513960
dtype: float64
Step 6.5 complete
I am running baseline 18 with ssp 5 of AGO
Problem
  Name                   :                 
  Objective sen

...

Step 6.5 complete
I am running baseline 18 with ssp 5 of TON
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 1514            
  Cones                  : 0               
  Scalar variables       : 1506            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Eliminator - tries                  : 0                 time                   : 0.00            
Lin. dep.  - tries                  : 0                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Optimizer terminated. Time: 0.03    


Interior-point solution summary
  Problem status  : PRIMAL_INFEASIBLE
  Solution status : PRIM

Step 6.5 complete
I am running baseline 18 with ssp 5 of TZA
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 4649            
  Cones                  : 0               
  Scalar variables       : 4641            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminat

Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 7626              after factor           : 7626            
Factor     - dense dim.             : 0                 flops                  : 7.18e+05        
ITE PFEAS    DFEAS    GFEAS    PRSTATUS   POBJ              DOBJ              MU       TIME  
0   4.4e+02  0.0e+00  2.0e+00  0.00e+00   5.000000000e-01   0.000000000e+00   1.0e+00  0.05  
1   4.1e+02  2.1e-13  1.9e+00  -8.57e-01  3.973532052e+03   3.890479520e+03   9.3e-01  0.08  
2   3.0e+02  1.5e-13  1.6e+00  -7.84e-01  6.867427383e+03   6.796072467e+03   6.8e-01  0.08  
3   2.7e+02  1.4e-13  1.4e+00  -5.66e-01  7.180011396e+03   7.111807635e+03   6.2e-01  0.09  
4   2.3e+02  1.3e-13  1.1e+00  -3.66e-01  1.127748858e+04   1.120752734e+04   5.2e-01  0.09  
5   1.7e+02  9.0e-14  6.7e-01  1.04e-02   2.949884921e+04   2.939608980e+04   3.8e-01  0.09  
6   8.5e+01  1.0e-13  2.2e-01  4.88e-01   7.6893

  Type                   : QO (quadratic optimization problem)
  Constraints            : 4311            
  Cones                  : 0               
  Scalar variables       : 4303            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 540
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 4836              conic                  : 4304            
Optimizer  - Semi-definite variables: 0                 scalarized             : 0               
Factor     - setup time             : 0.01              dense det. time        : 0.00            
Factor     - ML order time          : 0.00              GP order time          : 0.00            
Factor     - nonzeros before factor : 3446              after factor           : 3447            
Factor     - dense dim.           

Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 5210            
  Cones                  : 0               
  Scalar variables       : 5202            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer  - threads                : 4               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 15
Optimizer  - Cones                  : 1
Optimizer  - Scalar variables       : 5210              conic                  : 5203            
Optimizer  - Semi-definite variables: 0                 scalarized             : 0               
Factor     - setup time             : 0.00  

In [10]:
not_working

['KIR', 'SSD', 'SYC', 'TUV', 'XKX']

In [11]:
len(df_master)

67

In [12]:
df_master.to_csv('workers_income_BAJ.csv', index=False)

In [16]:
df_master = pd.read_csv('workers_income_BAJ.csv')

In [13]:
for country in not_working:
    finalhhframe = pd.read_csv('finalhhdataframes/{}_finalhhframe.csv'.format(country))
    print('=============={}============'.format(country))
    for col in finalhhframe.columns:
        if 'cat' in col:
            print(col, np.max(finalhhframe[col]))

==============KIR============
cat1workers 5.0
cat2workers 0.0
cat3workers 4.0
cat4workers 0.0
cat5workers 5.0
cat6workers 0.0
cat7workers 13.0
==============SSD============
cat1workers 7.0
cat2workers 7.0
cat3workers 17.0
cat4workers 5.0
cat5workers 9.0
cat6workers 9.0
cat7workers 26.0
==============SYC============
cat1workers 5.0
cat2workers 0.0
cat3workers 3.0
cat4workers 0.0
cat5workers 7.0
cat6workers 0.0
cat7workers 4.0
==============TUV============
cat1workers 8.0
cat2workers 0.0
cat3workers 0.0
cat4workers 0.0
cat5workers 6.0
cat6workers 0.0
cat7workers 9.0
==============XKX============
cat1workers 1.0
cat2workers 0.0
cat3workers 4.0
cat4workers 0.0
cat5workers 7.0
cat6workers 0.0
cat7workers 9.0


In [17]:
len(df_master)

67